In [8]:
library(tidyverse)

data <- read_csv("processed.cleveland.data", col_names = FALSE) |>
    rename(age = X1,
           sex = X2,
           chest_pain = X3,
           resting_blood_pressure = X4,
           cholesterol = X5,
           fasting_blood_sugar = X6, #1 if above 120mg/L
           resting_ecg = X7,
           max_heart_rate = X8,
           exercise_angina = X9,
           st_depression = X10,
           slope = X11,
           major_vessels = X12,
           thal = X13,
           diagnosis = X14)

# Leave only age, sex, chest pain, resting blood pressure, cholesterol, fasting blood sugar,
# max heart rate, exercise angina, diagnosis 

data_selected <- select(data, -resting_ecg, -st_depression, -slope, -major_vessels, -thal)

# change to categorical, only for preliminary data analysis in order for easier visualization:
# sex is categorical (1 = male; 0 = female)
# chest pain is categorical
    #   -- Value 1: typical angina
    #    -- Value 2: atypical angina
   #     -- Value 3: non-anginal pain
   #     -- Value 4: asymptomatic
# blood sugar is categorical
# anigina is categorical
# diagnosis is categorical

data_selected

# data is already tidy

# step 2:
# Using only training data, summarize the data in at least one table (this is exploratory data analysis).
# An example of a useful table could be one that reports the number of observations in each class,
# the means of the predictor variables you plan to use in your analysis and how many rows have missing data.

# step 3:
# Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do
# (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions
# of each of the predictor variables you plan to use in your analysis.

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,chest_pain,resting_blood_pressure,cholesterol,fasting_blood_sugar,max_heart_rate,exercise_angina,diagnosis
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
63,1,1,145,233,1,150,0,0
67,1,4,160,286,0,108,1,2
67,1,4,120,229,0,129,1,1
37,1,3,130,250,0,187,0,0
41,0,2,130,204,0,172,0,0
56,1,2,120,236,0,178,0,0
62,0,4,140,268,0,160,0,3
57,0,4,120,354,0,163,1,0
63,1,4,130,254,0,147,0,2
